In [ ]:
"""
Purpose

Shows how to use the AWS SDK for Python (Boto3) with the Amazon Kinesis API to
generate a data stream. This script generates data for several of the _Windows
and Aggregation_ examples in the Amazon Kinesis Data Analytics SQL Developer Guide.
"""

import datetime
import json
import random
import boto3
from botocore.config import Config

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
# Authentication
import AWS_credentials

session = boto3.Session(
    aws_access_key_id=AWS_credentials.ACCESS_KEY,
    aws_secret_access_key=AWS_credentials.SECRET_KEY,
    #aws_session_token=SESSION_TOKEN
)

In [ ]:
# see
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html

my_config = Config(
    region_name = 'us-east-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [ ]:
# set stream variable - note this stream has to be set up first in AWS
# see https://docs.aws.amazon.com/streams/latest/dev/tutorial-stock-data-kplkcl-create-stream.html
# a) set up Kinesis Stream
# b) create IAM Policy and attached Amazon Resource Name (ARN) for Kinesis Stream from a)
# c) create IAM user and attach Policy from b)


STREAM_NAME = "KinesisDataStream_demo"

##### Stock Price Stream

In [ ]:
def get_data():
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)}

def generate(stream_name, kinesis_client):
    while True:
        data = get_data()
        print(data) # print the whole streaming dictionary
        #print(data["PRICE"]) # print the stock prices
        plt.show()
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")
        
        

In [ ]:
if __name__ == '__main__':
    generate(STREAM_NAME, boto3.client('kinesis',     
                                       aws_access_key_id=AWS_credentials.ACCESS_KEY,
                                       aws_secret_access_key=AWS_credentials.SECRET_KEY,
                                       config=my_config))

##### Dymamic Chart

In [ ]:
def get_data():
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)}

def animate(i):

    #x = np.arange(0, 2*np.pi, 0.01)
    #line, = ax.plot(x, np.sin(x))

    #x = ['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']
    #line, = ax.plot(x, [100, 100,i,100,100])
    line.set_ydata([100, 100,i,100,100])  # update the data.
    return line,

def generate(stream_name, kinesis_client):
    while True:
        data = get_data()
        #print(data) # print the whole streaming dictionary
        #print(data["PRICE"]) # print the stock prices
        animation.FuncAnimation(fig, animate(data["PRICE"]), interval=20, blit=True, save_count=50)
        #plt.plot(data["PRICE"]) # chart
        plt.show(block = False)
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")
        
        

In [ ]:
# MUST RUN BELOW TO CHANGE backend to run an animation in the IPython console
# this will create a pop up live chart when generate function above called

%matplotlib qt

fig, ax = plt.subplots()

#x = np.arange(0, 2*np.pi, 0.01)
#line, = ax.plot(x, np.sin(x))

x = ['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']
line, = ax.plot(x, [100, 100,100,100,100])

In [ ]:
if __name__ == '__main__':
    generate(STREAM_NAME, boto3.client('kinesis',     
                                       aws_access_key_id=AWS_credentials.ACCESS_KEY,
                                       aws_secret_access_key=AWS_credentials.SECRET_KEY,
                                       config=my_config))

Reference:
    
    
https://docs.aws.amazon.com/code-samples/latest/catalog/python-kinesis-streams-dg_stockticker.py.html